In [25]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
import string

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, BatchNormalization,Dropout,SimpleRNN,Bidirectional,GRU
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import Adam
# import gensim 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import keras as k
from keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from keras.models import Sequential

In [26]:
df=pd.read_csv("IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [27]:
df=df.iloc[:10000,:] 
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [28]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [29]:
df['review']=df['review'].apply(lambda x: x.lower())

<ipython-input-29-70819ea598eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x: x.lower())


In [ ]:
df['review'] = df['review'].apply(lambda x: re.sub(r'([^\w\s])+', r'\1', x))

In [ ]:
for i in range(len(df['review'])):
            df['review'][i]=re.sub('<br />','',df['review'][i])         #Removing <br /> 
            df['review'][i] = re.sub(r'[^\w\s]', ' ', df['review'][i])  #Replace multiple punctuations with single 
            df['review'][i]=re.sub("[^\u0000-\u05C0\u2100-\u214F]+",'',df['review'][i]) # Removing non-English characters
            df['review'][i] = " ".join(df['review'][i] .split())   #removing extra spaces
           

In [ ]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
df['sentiment']=l.fit_transform(df['sentiment'])


In [33]:
X=df['review']

In [34]:
lab=df['sentiment']

In [35]:
lab=k.utils.to_categorical(df['sentiment'],dtype=int)
lab

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]])

In [36]:
text=df['review']

In [39]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [40]:
from nltk.tokenize import word_tokenize

In [41]:
voc_size = 4000                #Size of the vocabulary.
embedding_dim = 100              #The dimensionality of the word embeddings
maxlen = 0  
for i in range(len(text)):
    x = word_tokenize(text[i])     #Tokenize the cleaned review into words
    if len(x) > maxlen:        #finding maximum length
        maxlen = len(x)  
print("max length = {}".format(maxlen)) 
oov_tok = "<OOV>"                  # The out-of-vocabulary token, used to replace words that are not in the vocabulary

max length = 1873


In [42]:
tokenizer = Tokenizer(num_words=voc_size, oov_token=oov_tok)  #Tokenizer
tokenizer.fit_on_texts(text)                                    #Fit tokenizer on the 'Review' data
text_seq = tokenizer.texts_to_sequences(text)                   #Convert the data into sequences of integers using the tokenizer

In [43]:
X = pad_sequences(text_seq, maxlen=maxlen, padding='pre')
X

array([[   0,    0,    0, ...,  125, 3816,  484],
       [   0,    0,    0, ..., 2159,   70,  216],
       [   0,    0,    0, ...,   66,   18,  357],
       ...,
       [   0,    0,    0, ...,   17,  335,  156],
       [   0,    0,    0, ...,   51,   19,   71],
       [   0,    0,    0, ...,    7,  972, 2673]], dtype=int32)

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, lab, train_size=0.8,shuffle=True)

In [45]:
print("Training Data Review:",len(X_train))
print("Test Data Review:",len(X_test))
print("Training Data Labels:",len(y_train))
print("Test Data Labels:",len(y_test))

Training Data Review: 8000
Test Data Review: 2000
Training Data Labels: 8000
Test Data Labels: 2000


## Models 

In [46]:
# define an early stopping 
early_stop = EarlyStopping(monitor='val_loss', patience=3) 

## RNN

In [47]:

RNNmodel = Sequential()                                               #Creating a Sequential model
RNNmodel.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
RNNmodel.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
RNNmodel.add(Dropout(0.5))                                            #Dropout    
RNNmodel.add(SimpleRNN(32,kernel_regularizer=regularizers.l2(0.01)))                       #Add a SimpleRNN layer to the model with 32 units
RNNmodel.add(Dense(2, activation='softmax'))                          #Add a Softmax layer 

RNNmodel.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

RNNmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1873, 100)         400000    
                                                                 
 batch_normalization (BatchN  (None, 1873, 100)        400       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1873, 100)         0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                4256      
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 404,722
Trainable params: 404,522
Non-trainable params: 200
________________________________________________

In [48]:

RNNmodel.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 35s 2s/step - loss: 1.1938 - accuracy: 0.5262 - val_loss: 1.0950 - val_accuracy: 0.5430
Epoch 2/5
12/12 [==============================] - 33s 3s/step - loss: 1.0835 - accuracy: 0.5715 - val_loss: 1.0452 - val_accuracy: 0.5485
Epoch 3/5
12/12 [==============================] - 31s 3s/step - loss: 0.9998 - accuracy: 0.6202 - val_loss: 1.0005 - val_accuracy: 0.5605
Epoch 4/5
12/12 [==============================] - 28s 2s/step - loss: 0.9341 - accuracy: 0.6462 - val_loss: 0.9619 - val_accuracy: 0.5665
Epoch 5/5
12/12 [==============================] - 30s 3s/step - loss: 0.8677 - accuracy: 0.6797 - val_loss: 0.9261 - val_accuracy: 0.5755


In [49]:
# Evaluating the model on the test data
RNN_Loss, RNN_accuracy = RNNmodel.evaluate(X_test, y_test)
print('RNN Accuracy:', RNN_accuracy)


63/63 [==============================] - 8s 132ms/step - loss: 0.9288 - accuracy: 0.5465
RNN Accuracy: 0.546500027179718


## LSTM

In [50]:

LSTMmodel = Sequential()                                               #Creating a Sequential model
LSTMmodel.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
LSTMmodel.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
LSTMmodel.add(Dropout(0.5))                                            #Dropout    
LSTMmodel.add(LSTM(32, activation='relu',))                            #Add a LSTM layer to the model with 32 units
LSTMmodel.add(Dense(2, activation='softmax'))                          #Add a Softmax layer 

LSTMmodel.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) #compile

LSTMmodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_1 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 1873, 100)         0         
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 417,490
Trainable params: 417,290
Non-trainable params: 200
______________________________________________

In [51]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)  
LSTMmodel.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 51s 4s/step - loss: 0.7337 - accuracy: 0.5378 - val_loss: 0.6923 - val_accuracy: 0.5715
Epoch 2/5
12/12 [==============================] - 47s 4s/step - loss: 0.6448 - accuracy: 0.6272 - val_loss: 0.6913 - val_accuracy: 0.6215
Epoch 3/5
12/12 [==============================] - 49s 4s/step - loss: 0.5839 - accuracy: 0.6910 - val_loss: 0.6904 - val_accuracy: 0.5690
Epoch 4/5
12/12 [==============================] - 48s 4s/step - loss: 0.5169 - accuracy: 0.7462 - val_loss: 0.6893 - val_accuracy: 0.5650
Epoch 5/5
12/12 [==============================] - 47s 4s/step - loss: 0.4907 - accuracy: 0.7713 - val_loss: 0.6884 - val_accuracy: 0.5630


In [52]:
# Evaluating the model on the test data
LSTM_Loss, LSTM_accuracy = LSTMmodel.evaluate(X_test, y_test)
print('LSTM Accuracy:', LSTM_accuracy)


63/63 [==============================] - 15s 242ms/step - loss: 0.6880 - accuracy: 0.5765
LSTM Accuracy: 0.5764999985694885


## GRU

In [53]:

GRUmodel = Sequential()                                               #Creating a Sequential model
GRUmodel.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
GRUmodel.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
GRUmodel.add(Dropout(0.5))                                            #Dropout    
GRUmodel.add(GRU(8, dropout=0.25, 
                 recurrent_dropout=0.25,
                 )) #Add a GRU layer to the model with 32 units

GRUmodel.add(Dense(2, activation='softmax'))  #Add a Softmax layer 

GRUmodel.compile(loss='binary_crossentropy',
                  optimizer='rmsprop', metrics=['accuracy']) #compile

GRUmodel.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_2 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 1873, 100)         0         
                                                                 
 gru (GRU)                   (None, 8)                 2640      
                                                                 
 dense_2 (Dense)             (None, 2)                 18        
                                                                 
Total params: 403,058
Trainable params: 402,858
Non-trainable params: 200
______________________________________________

In [54]:

GRUmodel.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=1024,callbacks=[early_stop])

Epoch 1/5
6/6 [==============================] - 54s 8s/step - loss: 0.7564 - accuracy: 0.5162 - val_loss: 0.6917 - val_accuracy: 0.5395
Epoch 2/5
6/6 [==============================] - 49s 8s/step - loss: 0.7179 - accuracy: 0.5500 - val_loss: 0.6909 - val_accuracy: 0.5720
Epoch 3/5
6/6 [==============================] - 46s 8s/step - loss: 0.6936 - accuracy: 0.5732 - val_loss: 0.6900 - val_accuracy: 0.5815
Epoch 4/5
6/6 [==============================] - 48s 8s/step - loss: 0.6727 - accuracy: 0.6053 - val_loss: 0.6891 - val_accuracy: 0.5780
Epoch 5/5
6/6 [==============================] - 48s 8s/step - loss: 0.6544 - accuracy: 0.6368 - val_loss: 0.6879 - val_accuracy: 0.5935


In [55]:
# Evaluating the model on the test data
GRU_Loss, GRU_accuracy = GRUmodel.evaluate(X_test, y_test)
print('GRU Accuracy:', GRU_accuracy)


63/63 [==============================] - 34s 547ms/step - loss: 0.6877 - accuracy: 0.6015
GRU Accuracy: 0.6014999747276306


## Bi-Directional RNN

In [56]:


BiRNN = Sequential()                                               #Creating a Sequential model
BiRNN.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
BiRNN.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
BiRNN.add(Dropout(0.5))                                            #Dropout    
BiRNN.add(Bidirectional(SimpleRNN(32, activation='relu',)))        #Add a Bidirectional SimpleRNN layer to the model with 32 units
BiRNN.add(Dense(2, activation='softmax'))                          #Add a Softmax layer 

BiRNN.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

BiRNN.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_3 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 1873, 100)         0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               8512      
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 409,042
Trainable params: 408,842
Non-tr

In [57]:

BiRNN.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 52s 4s/step - loss: 0.8112 - accuracy: 0.5075 - val_loss: 0.6928 - val_accuracy: 0.5025
Epoch 2/5
12/12 [==============================] - 49s 4s/step - loss: 0.7017 - accuracy: 0.5768 - val_loss: 0.6922 - val_accuracy: 0.5025
Epoch 3/5
12/12 [==============================] - 48s 4s/step - loss: 0.6718 - accuracy: 0.5953 - val_loss: 0.6913 - val_accuracy: 0.5050
Epoch 4/5
12/12 [==============================] - 47s 4s/step - loss: 0.6360 - accuracy: 0.6410 - val_loss: 0.6899 - val_accuracy: 0.5075
Epoch 5/5
12/12 [==============================] - 48s 4s/step - loss: 0.5956 - accuracy: 0.6830 - val_loss: 0.6895 - val_accuracy: 0.5040


In [58]:
# Evaluating the model on the test data
BiRNN_Loss, BiRNN_accuracy = BiRNN.evaluate(X_test,y_test)
print('Bi-Directional RNN Accuracy:', BiRNN_accuracy)

63/63 [==============================] - 18s 287ms/step - loss: 0.6888 - accuracy: 0.5105
Bi-Directional RNN Accuracy: 0.5105000138282776


## Bi-Directional LSTM

In [59]:

BiLSTM = Sequential()                                               #Creating a Sequential model
BiLSTM.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
BiLSTM.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
BiLSTM.add(Dropout(0.5))                                            #Dropout    
BiLSTM.add(Bidirectional(LSTM(32, activation='relu',)))                         #Add a Bi-Directional LSTM layer to the model with 32 units
BiLSTM.add(Dense(2, activation='softmax'))                          #Add a Softmax layer 

BiLSTM.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) #compile

BiLSTM.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_4 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 1873, 100)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 130       
                                                                 
Total params: 434,578
Trainable params: 434,378
Non-tr

In [60]:

BiLSTM.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 91s 7s/step - loss: 481574032697720832.0000 - accuracy: 0.4932 - val_loss: 0.6963 - val_accuracy: 0.5025
Epoch 2/5
12/12 [==============================] - 87s 7s/step - loss: nan - accuracy: 0.5048 - val_loss: nan - val_accuracy: 0.4975
Epoch 3/5
12/12 [==============================] - 91s 8s/step - loss: nan - accuracy: 0.4992 - val_loss: nan - val_accuracy: 0.4975
Epoch 4/5
12/12 [==============================] - 88s 7s/step - loss: nan - accuracy: 0.4992 - val_loss: nan - val_accuracy: 0.4975


In [61]:
# Evaluating the model on the test data
BiLSTM_Loss, BiLSTM_accuracy = BiLSTM.evaluate(X_test,y_test)
print('Bi-Directional LSTM Accuracy:', BiLSTM_accuracy)

63/63 [==============================] - 30s 481ms/step - loss: nan - accuracy: 0.4910
Bi-Directional LSTM Accuracy: 0.4909999966621399


## Bi-Directional GRU

In [62]:

BiGRU = Sequential()                                               #Creating a Sequential model
BiGRU.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
BiGRU.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
BiGRU.add(Dropout(0.5))                                            #Dropout    
BiGRU.add(Bidirectional(GRU(32, dropout=0.25, 
                 recurrent_dropout=0.25,
                 activation='relu',)) )#Add a GRU layer to the model with 32 units

BiGRU.add(Dense(2, activation='softmax'))  #Add a Softmax layer 

BiGRU.compile(loss='binary_crossentropy',
                  optimizer='rmsprop', metrics=['accuracy']) #compile

BiGRU.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_5 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 1873, 100)         0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               25728     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 426,258
Trainable params: 426,058
Non-tr

In [63]:

BiGRU.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 183s 15s/step - loss: 0.7423 - accuracy: 0.5240 - val_loss: 0.6926 - val_accuracy: 0.5035
Epoch 2/5
12/12 [==============================] - 183s 15s/step - loss: 0.6891 - accuracy: 0.5790 - val_loss: 0.6920 - val_accuracy: 0.5050
Epoch 3/5
12/12 [==============================] - 180s 15s/step - loss: 0.6629 - accuracy: 0.6160 - val_loss: 0.6910 - val_accuracy: 0.5265
Epoch 4/5
12/12 [==============================] - 178s 15s/step - loss: 0.6323 - accuracy: 0.6535 - val_loss: 0.6896 - val_accuracy: 0.5455
Epoch 5/5
12/12 [==============================] - 181s 15s/step - loss: 0.6018 - accuracy: 0.6808 - val_loss: 0.6878 - val_accuracy: 0.5820


In [64]:
# Evaluating the model on the test data
BiGRU_Loss, BiGRU_accuracy = BiGRU.evaluate(X_test,y_test)
print('Bi-Directional GRU Accuracy:', BiGRU_accuracy)

63/63 [==============================] - 58s 924ms/step - loss: 0.6876 - accuracy: 0.5765
Bi-Directional GRU Accuracy: 0.5764999985694885


## Stacked Simple RNN

In [65]:

SRNN = Sequential()                                               #Creating a Sequential model
SRNN.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
SRNN.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
SRNN.add(Dropout(0.5))                                            #Dropout    
SRNN.add(SimpleRNN(32, activation='relu',return_sequences=True))                       #Add a SimpleRNN layer to the model with 32 units
SRNN.add(SimpleRNN(32))
SRNN.add(Dense(2, activation='softmax'))                          #Add a Softmax layer 

SRNN.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

SRNN.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_6 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 1873, 100)         0         
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 1873, 32)          4256      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 2)                 66        
                                                      

In [66]:

SRNN.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 55s 4s/step - loss: 0.7397 - accuracy: 0.5292 - val_loss: 0.7030 - val_accuracy: 0.5025
Epoch 2/5
12/12 [==============================] - 53s 4s/step - loss: 0.6893 - accuracy: 0.5718 - val_loss: 0.6938 - val_accuracy: 0.5070
Epoch 3/5
12/12 [==============================] - 53s 4s/step - loss: 0.6519 - accuracy: 0.6320 - val_loss: 0.6928 - val_accuracy: 0.5090
Epoch 4/5
12/12 [==============================] - 53s 4s/step - loss: 0.6174 - accuracy: 0.6712 - val_loss: 0.7042 - val_accuracy: 0.5025
Epoch 5/5
12/12 [==============================] - 50s 4s/step - loss: 0.5913 - accuracy: 0.6875 - val_loss: 0.6963 - val_accuracy: 0.5060


In [67]:
# Evaluating the model on the test data
SRNN_Loss, SRNN_accuracy = SRNN.evaluate(X_test, y_test)
print('Stacked RNN Test Accuracy:', SRNN_accuracy)

63/63 [==============================] - 18s 292ms/step - loss: 0.6948 - accuracy: 0.5100
Stacked RNN Test Accuracy: 0.5099999904632568


## Stacked LSTM

In [68]:

SLSTM = Sequential()                                               #Creating a Sequential model
SLSTM.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
SLSTM.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
SLSTM.add(LSTM(32, dropout=0.25, recurrent_dropout=0.25, return_sequences=True,))                            #Add a LSTM layer to the model with 32 units
SLSTM.add(LSTM(32,dropout=0.25, recurrent_dropout=0.25,)) 
SLSTM.add(Dense(2, activation='softmax'))                           #Add a Softmax layer 

SLSTM.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) #compile

SLSTM.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_7 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 1873, 32)          17024     
                                                                 
 lstm_3 (LSTM)               (None, 32)                8320      
                                                                 
 dense_7 (Dense)             (None, 2)                 66        
                                                                 
Total params: 425,810
Trainable params: 425,610
Non-trainable params: 200
______________________________________________

In [69]:

SLSTM.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 206s 17s/step - loss: 0.6888 - accuracy: 0.5430 - val_loss: 0.6922 - val_accuracy: 0.5205
Epoch 2/5
12/12 [==============================] - 198s 17s/step - loss: 0.6570 - accuracy: 0.6110 - val_loss: 0.6893 - val_accuracy: 0.5480
Epoch 3/5
12/12 [==============================] - 202s 17s/step - loss: 0.5921 - accuracy: 0.6890 - val_loss: 0.6845 - val_accuracy: 0.5940
Epoch 4/5
12/12 [==============================] - 200s 17s/step - loss: 0.5366 - accuracy: 0.7343 - val_loss: 0.6800 - val_accuracy: 0.6510
Epoch 5/5
12/12 [==============================] - 198s 16s/step - loss: 0.4652 - accuracy: 0.7903 - val_loss: 0.6733 - val_accuracy: 0.6285


In [70]:
# Evaluating the model on the test data
SLSTM_Loss, SLSTM_accuracy = SLSTM.evaluate(X_test, y_test)
print('Stacked LSTM Accuracy:', SLSTM_accuracy)

63/63 [==============================] - 60s 959ms/step - loss: 0.6722 - accuracy: 0.6400
Stacked LSTM Accuracy: 0.6399999856948853


## Stacked GRU

In [71]:

SGRU = Sequential()                                               #Creating a Sequential model
SGRU.add(Embedding(voc_size, embedding_dim, input_length=maxlen)) #Add an embedding layer to the model 
SGRU.add(BatchNormalization())                                    #BatchNormalization layer to normalize the activations of the previous layer       
SGRU.add(Dropout(0.5))                                            #Dropout    
SGRU.add(GRU(8, dropout=0.25, 
                 recurrent_dropout=0.25,
                 return_sequences=True
                 )) #Add a GRU layer to the model with 32 units
SGRU.add(GRU(8, dropout=0.2, recurrent_dropout=0.2))

SGRU.add(Dense(2, activation='softmax'))  #Add a Softmax layer 

SGRU.compile(loss='binary_crossentropy',
                  optimizer='rmsprop', metrics=['accuracy']) #compile

SGRU.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1873, 100)         400000    
                                                                 
 batch_normalization_8 (Batc  (None, 1873, 100)        400       
 hNormalization)                                                 
                                                                 
 dropout_7 (Dropout)         (None, 1873, 100)         0         
                                                                 
 gru_2 (GRU)                 (None, 1873, 8)           2640      
                                                                 
 gru_3 (GRU)                 (None, 8)                 432       
                                                                 
 dense_8 (Dense)             (None, 2)                 18        
                                                      

In [72]:

SGRU.fit(X_train,y_train,validation_split=0.25,epochs=5,batch_size=512,callbacks=[early_stop])

Epoch 1/5
12/12 [==============================] - 186s 15s/step - loss: 0.6970 - accuracy: 0.5188 - val_loss: 0.6922 - val_accuracy: 0.5155
Epoch 2/5
12/12 [==============================] - 180s 15s/step - loss: 0.6846 - accuracy: 0.5633 - val_loss: 0.6911 - val_accuracy: 0.5325
Epoch 3/5
12/12 [==============================] - 184s 15s/step - loss: 0.6731 - accuracy: 0.5982 - val_loss: 0.6894 - val_accuracy: 0.5780
Epoch 4/5
12/12 [==============================] - 180s 15s/step - loss: 0.6543 - accuracy: 0.6410 - val_loss: 0.6868 - val_accuracy: 0.5935
Epoch 5/5
12/12 [==============================] - 179s 15s/step - loss: 0.6313 - accuracy: 0.6753 - val_loss: 0.6833 - val_accuracy: 0.6335


In [73]:
# Evaluating the model on the test data
SGRU_Loss, SGRU_accuracy = SGRU.evaluate(X_test, y_test)
print('Stacked LSTM Accuracy:', SGRU_accuracy)

63/63 [==============================] - 68s 1s/step - loss: 0.6833 - accuracy: 0.6450
Stacked LSTM Accuracy: 0.6449999809265137


ALL MODEL COMPARISON

In [80]:
print('Models\t\t\tAccuracy\t\t\tLoss')
print('RNN\t\t',RNN_accuracy,'\t\t',RNN_Loss,)
print('LSTM\t\t',LSTM_accuracy,'\t\t',LSTM_Loss,)
print('GRU\t\t',GRU_accuracy,'\t\t',GRU_Loss,)
print('Bi-RNN\t\t',BiRNN_accuracy,'\t\t',BiRNN_Loss,)
print('Bi-LSTM\t\t',BiLSTM_accuracy,'\t\t',BiLSTM_Loss,)
print('Bi-GRU\t\t',BiGRU_accuracy,'\t\t',BiGRU_Loss,)
print('Stacked RNN\t',SRNN_accuracy,'\t\t',SRNN_Loss,)
print('Stacked LSTM\t',SLSTM_accuracy,'\t\t',SLSTM_Loss,)
print('Stacked GRU\t',SGRU_accuracy,'\t\t',SGRU_Loss,)
print('Glove-LSTM\t',0.6769999861717224,'\t\t',0.7718069553375244,)  #implemented glove in seperate .ipynb file

Models			Accuracy			Loss
RNN		 0.546500027179718 		 0.9288047552108765
LSTM		 0.5764999985694885 		 0.6879608631134033
GRU		 0.6014999747276306 		 0.6876621246337891
Bi-RNN		 0.5105000138282776 		 0.6888228058815002
Bi-LSTM		 0.4909999966621399 		 nan
Bi-GRU		 0.5764999985694885 		 0.6875886917114258
Stacked RNN	 0.5099999904632568 		 0.6948187947273254
Stacked LSTM	 0.6399999856948853 		 0.6722079515457153
Stacked GRU	 0.6449999809265137 		 0.6833115220069885
Glove-LSTM	 0.6769999861717224 		 0.7718069553375244
